In [ ]:
from langchain.vectorstores import Weaviate
import weaviate
import locale
# Specify embedding model (using Huggingface sentence transformer)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain import HuggingFaceHub
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Fixing unicode error in google colab
locale.getpreferredencoding = lambda: "UTF-8"

In [1]:
%pip install weaviate-client tiktoken pypdf rapidocr-onnxruntime sentence-transformers

^C
Note: you may need to restart the kernel to use updated packages.


   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.3 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.3 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.3 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.3 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.3 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.3 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.3 MB ? eta -:--:--
   -- ----------------------------

ERROR: Could not install packages due to an OSError: HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Max retries exceeded with url: /packages/96/f2/25b27b396af03d5b64e61976b14f7209e2939e9e806c10749b6d277c273e/transformers-4.52.4-py3-none-any.whl (Caused by NewConnectionError('<pip._vendor.urllib3.connection.HTTPSConnection object at 0x000002BA06FF4F10>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))



In [ ]:
WEAVIATE_API_KEY = "WEAVIATE_API_KEY"
WEATIATE_CLUSTER = "WEATIATE_CLUSTER"                          

In [ ]:
client = weaviate.Client(
    url = WEATIATE_CLUSTER, auth_client_secret = weaviate.AutoApiKey(WEAVIATE_API_KEY)
)

In [ ]:
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"

# Model_kwargs = {"device":"cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    #model_kwargs=model_kwargs
)

In [ ]:
loader = PyPDFLoader("Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf", extract_images=True)
pages = loader.load()

Creating Chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(pages)

In [ ]:
vector_db = Weaviate.from_documents(
    docs, embeddings, client=client, by_text=False
)

Similarity search

In [ ]:
print(
    vector_db.similarity_search(
        "What is attention?", k=3
    )[0].page_content
)

In [ ]:
template = """ 

You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the quetsion.
If you do not know the answer, just say that you do not know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
 """

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

Loading Model

In [ ]:
huggingfacehub_api_token = "HUGGINGFACEHUB_API_TOKEN"

model = HuggingFaceHub(
    huggingfacehub_api_token = huggingfacehub_api_token,
    repo_id = "mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs = {"temprature":1, "max_length":180}
)

In [ ]:
retriever = vector_db.as_retriever()

In [ ]:
output_parser = StrOutputParser()

In [ ]:
rag_chain = (
    {"context": retriever, "question":RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [ ]:
rag_chain.invoke("What is RAG application?")